<a href="https://colab.research.google.com/github/aelydens/aie-resources/blob/main/OpenAI_Assistants_Building_Agentic_RAG_with_Function_Calling_API_and_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI Assistants - Building Agentic RAG with the Function Calling, Retrieval, and Code Interpreter Tools

Today we'll explore using OpenAI's Python SDK to create, manage, and use the OpenAI Assistant API!

## Dependencies

We'll start, as we usually do, with some dependiencies and our API key!

In [59]:
!pip install -qU openai

In [60]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

OpenAI API Key:··········


## Simple Assistant

Let's create a simple Assistant to understand more about how the API works to start!

### OpenAI Client

At the core of the OpenAI Python SDK is the Client!

> NOTE: For ease of use, we'll start with the synchronous `OpenAI()`. OpenAI does provide an `AsyncOpenAI()` that you could leverage as well!

In [61]:
from openai import OpenAI

client = OpenAI()

### Creating An Assistant

Leveraging what we know about the OpenAI API from previous sessions - we're going to start by simply initializing an Assistant.

Before we begin, we need to think about a few customization options we have:

- `name` - Straight forward enough, this is what our Assistant's name will be
- `instructions` - similar to a system message, but applied at an Assistant level, this is how we can guide the Assistant's tone, behaviour, functionality, and more!
- `model` - this will allow us to choose which model we would prefer to use for our Assistant

Let's start by setting some instructions for our Assistant.



In [62]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown Fill out the fields below to add your Assistant's name, instructions, and desired model!

name = "Sous Chef" # @param {type: "string"}
instructions = "You are a world-class chef and love discussing cooking. If a user gives you ingredients, generate a recipe." # @param {type: "string"}
model = "gpt-3.5-turbo" # @param ["gpt-3.5-turbo", "gpt-4-turbo-preview", "gpt-4"]

### Initialize Assistant

Now that we have our desired name, instruction, and model - we can initialize our Assistant!

In [63]:
assistant = client.beta.assistants.create(
    name=name,
    instructions=instructions,
    model=model,
)

Let's examine our `assistant` object and see what we find!

In [64]:
assistant

Assistant(id='asst_ddE5WYbhOVyjblkmyysSUJMb', created_at=1708813474, description=None, file_ids=[], instructions='You are a world-class chef and love discussing cooking. If a user gives you ingredients, generate a recipe.', metadata={}, model='gpt-3.5-turbo', name='Sous Chef', object='assistant', tools=[])

There are a number of useful parameters here, but we'll call out a few:

- `id` - since we may have multiple Assistant's, knowing which Assistant we're interacting with will help us ensure the desired user experience!
- `description` - A natrual language description of our Assistant could help others understand what it's supposed to do!
- `file_ids` - if we wanted to use the Retrieval tool, this would let us know what files we had given our Assistant

### Creating a Thread

Behind the scenes our Assistant is powered by the idea of "threads".

You can think of threads as individual conversations that interact with the Assistant.

Let's create a thread now!

In [65]:
thread = client.beta.threads.create()

Let's look at our `thread` object.

In [66]:
thread

Thread(id='thread_CjhPzYcp3OBGd5e55VILgXTk', created_at=1708813474, metadata={}, object='thread')

Notice some key attributes:

- `id` - since each Thread is like a conversation, we need some way to specify which thread we're dealing with when interacting with them
- `tool_resources` - this will become more relevant as we add tools since we'll need a way to verify which tools we have access to when interacting with our Assistant

### Adding Messages to Our Thread

Now that we have our Thread (or conversation) we can start adding messages to it!

Let's add a simple message that asks about how our Assistant is feeling.

Notice the parameters we're leveraging:

- `thread_id` - since each Thread is like a conversation, we need some way to address a specific conversation. We can use `thread.id` to do this.
- `role` - similar to when we used our chat completions endpoint, this parameter specifies who the message is coming from. You can leverage this in the same ways you would through the chat completions endpoint.
- `content` - this is where we can place the actual text our Assistant will interact with

> NOTE: Feel free to substitute a relevant message based on the Assistant you created

In [67]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"How old are you?"
)

Again, let's examine our `message` object!

In [68]:
message

ThreadMessage(id='msg_0oUsjmW0TkxRWHQJbeEwhQ4H', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='How old are you?'), type='text')], created_at=1708813474, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_CjhPzYcp3OBGd5e55VILgXTk')

### Running Our Thread

Now that we have an Assistant, and we've given that Assistant a Thread, and we've added a Message to that Thread - we're ready to run our Assistant!

Notice that this process lets us add (potentially) multiple messages to our Assistant. We can leverage that behaviour for few/many-shot examples, and more!

In [69]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown We can also override the Assistant's instructions when we run a thread.

# @markdown Use one of the [Prompt Principles for Instruction](https://arxiv.org/pdf/2312.16171v1.pdf) to improve the likeliehood of a correct or valuable response from your Assistant.

additional_instructions = "I’m going to tip $50K for a better solution! " # @param {type: "string"}

Let's run our Thread!

In [70]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=instructions + " " + additional_instructions
)

Now that we've run our thread, let's look at the object!

In [71]:
run

Run(id='run_konvz3FKvWZi9CtUTnae3ueJ', assistant_id='asst_ddE5WYbhOVyjblkmyysSUJMb', cancelled_at=None, completed_at=None, created_at=1708813474, expires_at=1708814074, failed_at=None, file_ids=[], instructions='You are a world-class chef and love discussing cooking. If a user gives you ingredients, generate a recipe. I’m going to tip $50K for a better solution! ', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_CjhPzYcp3OBGd5e55VILgXTk', tools=[], usage=None)

Notice we have access to a few very powerful parameters in this `run` object.

- `completed_at` - this will help us determine when we can expect to retrieve a response
- `failed_at` - this can highlight any issues our run ran into
- `status` - is another way we can understand how the flow is going

### Retrieving Our Run

Now that we've created our run, let's retrieve it.

We're going to wrap this in a simple loop to make sure we're not retrieving it too early.

In [72]:
import time

while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

In [73]:
print(run.status)

completed


Now that our run is completed - we can retieve the messages from our thread!

Notice that our run helps us understand how things are going - but it isn't where we're going to find our responses or messages. Those are added on the backend into our thread.

This leads to a simple, but important, flow:

1. We add messages to a thread.
2. We create a run on that thread.
3. We wait until the run is finished.
4. We check our thread for the new messages.

### Checking Our Thread

Now we can get a list of messages from our thread!

In [74]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [75]:
messages.data[0]

ThreadMessage(id='msg_3JANEZ14jiwmRdHlUGrn6Xdq', assistant_id='asst_ddE5WYbhOVyjblkmyysSUJMb', content=[MessageContentText(text=Text(annotations=[], value='I’m a computer program, so I don’t have an age. But I’m here to assist you with any cooking-related questions or tasks you have! If you’d like a recipe or cooking tips, feel free to ask.'), type='text')], created_at=1708813475, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_konvz3FKvWZi9CtUTnae3ueJ', thread_id='thread_CjhPzYcp3OBGd5e55VILgXTk')

## Adding Tools

Now that we have an understanding of how Assistant works, we can start thinking about adding tools.

We'll go through 3 separate tools and explore how we can leverage them!

Let's start with the most familiar tool - the Retriever!


### Creating an Assistant with the Retriever Tool

The first thing we'll want to do is create an assistant with the Retriever tool.

This is also going to require some data. We'll provided data - but you're very much encouraged to use your own files to explore how the Assistant works for your use case.

#### Collect and Add Data

First, we need some data. Second, we need to add the data to our Assistant!

Let's start with grabbing some data!

In [92]:
!wget https://www.gutenberg.org/files/84/84-h/84-h.htm -O frankenstein.html

--2024-02-24 22:25:54--  https://www.gutenberg.org/files/84/84-h/84-h.htm
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 466919 (456K) [text/html]
Saving to: ‘frankenstein.html’

frankenstein.html   100%[===================>] 455.98K  --.-KB/s    in 0.09s   

2024-02-24 22:25:55 (4.79 MB/s) - ‘frankenstein.html’ saved [466919/466919]



Now we can upload our file!

Pay attention to [this](https://platform.openai.com/docs/assistants/tools/supported-files) documentation to see what kinds of files can be uploaded.

> NOTE: Per the OpenAI [docs](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) The maximum file size is 512 MB and no more than 2,000,000 tokens (computed automatically when you attach a file)

In [93]:
file_reference = client.files.create(
  file=open("frankenstein.html", "rb"),
  purpose='assistants'
)

Let's look at what our `file_reference` contains!

In [94]:
file_reference

FileObject(id='file-Ssn6ZpKFKlTaFeNFXcxIYE8k', bytes=466919, created_at=1708813557, filename='frankenstein.html', object='file', purpose='assistants', status='processed', status_details=None)

#### Create and Use Assistant

Now that we have our file - we can attach it to an Assistant, and we can give that Assistant the ability to use it for retrieval through the Retrieval tool!

> NOTE: Please pay attention to [pricing](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) and don't forget to delete your files when you're done!

In [95]:
assistant = client.beta.assistants.create(
  name=name + "+ Retrieval",
  instructions=instructions,
  model=model,
  tools=[{"type": "retrieval"}],
  file_ids=[file_reference.id]
)

Let's try submitting a message to our Assistant and seeing what kind of answer we get!

We'll outline the steps needed to do this in full:

1. Create an Assistant
2. Create a Thread
3. Add Messages to that Thread
4. Create a Run on that Thread
5. Wait for Run to Complete
6. Collect Messages from the Thread

Let's do that below!

In [96]:
# Create a Thread
thread = client.beta.threads.create()

# Add Messages to that Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"What is the first words Victor Frankenstein speaks?"
)

# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress
in_progress


Let's look at the final result!

In [97]:
messages

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_bD1jH0jvSBLqtZkTi2AYQ8Al', assistant_id='asst_rGOznerMpIBxM0xAGUv2hPsQ', content=[MessageContentText(text=Text(annotations=[TextAnnotationFileCitation(end_index=218, file_citation=TextAnnotationFileCitationFileCitation(file_id='file-Ssn6ZpKFKlTaFeNFXcxIYE8k', quote='PE html P'), start_index=208, text='【7†source】', type='file_citation')], value='The first words spoken by Victor Frankenstein in the book are: "You will rejoice to hear that no disaster has accompanied the commencement of an enterprise which you have regarded with such evil forebodings."【7†source】.'), type='text')], created_at=1708813573, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_PZvimFigd4xwgsTucSg5ixO1', thread_id='thread_MbU9y0snomXIwYm7kKVDfmaf'), ThreadMessage(id='msg_DLZzkn6XokwPnvuQoSQMfMTr', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='What is the first words Victor Frankenstein speaks?

Let's do some clean up to make sure we're not being charged anything extra by deleting our resources.

In [82]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)

### Creating an Assistant with the Code Interpreter Tool

Now that we've explored the Retrieval Tool - let's try the Code Interpreter tool!

The process will be almost exactly the same - but we can explore a different query, and we'll add our file at the Message level!

In [98]:
assistant = client.beta.assistants.create(
  name=name + "+ Code Interpreter",
  instructions=instructions,
  model=model,
  tools=[{"type": "code_interpreter"}],
)

In the following example, we'll also see how we can package the Thread creation with the Message adding step!

> NOTE: Files added at the message/thread level will not be available to the Assistant outside of that Thread.

In [99]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What kind of file is this?",
      "file_ids": [file_reference.id]
    }
  ]
)

> NOTE: Remember that we create runs at the *thread* level - and so don't need the message object to continue.

In [100]:
# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress


We can check the specific steps that the Code Interpreter ran to figure out what steps the Assistant took!

In [101]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

In [102]:
for step in run_steps.data:
  print(step.step_details)

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_ET9G5Jdu81YLOs0mbHjwKKIY'), type='message_creation')
ToolCallsStepDetails(tool_calls=[CodeToolCall(id='call_WkqUiXjiF8vn0YQWVDmhFZHA', code_interpreter=CodeInterpreter(input='# Read the first few lines of the file to identify its content\r\nwith open("/mnt/data/file-Ssn6ZpKFKlTaFeNFXcxIYE8k", \'r\') as file:\r\n    first_few_lines = [next(file) for _ in range(5)]\r\n\r\nfirst_few_lines', outputs=[CodeInterpreterOutputLogs(logs='[\'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN"\\n\',\n \'"http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\\n\',\n \'<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">\\n\',\n \'<head>\\n\',\n \'<meta http-equiv="Content-Type" content="text/html;charset=utf-8" />\\n\']', type='logs')]), type='code_interpreter')], type='tool_calls')
MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_BulPUDbmLtDcXiLdjE1ywxpS'), type='message_creatio

In [103]:
messages

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_ET9G5Jdu81YLOs0mbHjwKKIY', assistant_id='asst_TCdRVqKYgXZAJygxKfI0W5OC', content=[MessageContentText(text=Text(annotations=[], value="The initial lines of the file suggest that it is an HTML file. Since it is an HTML file, I won't be able to generate a recipe based on its content. If you have any specific ingredients you'd like to share or if there's anything else you'd like to know or do, feel free to let me know!"), type='text')], created_at=1708813666, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_6DbV7XgFwGSCd5bx0evONXBZ', thread_id='thread_nROwqYKMQwoKcQyAh6G1PStQ'), ThreadMessage(id='msg_BulPUDbmLtDcXiLdjE1ywxpS', assistant_id='asst_TCdRVqKYgXZAJygxKfI0W5OC', content=[MessageContentText(text=Text(annotations=[], value='It seems there was an error in determining the file type based on the extension. Let me try to read the first few lines of the file directly to identify its content.'), typ

In [104]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)

NotFoundError: Error code: 404 - {'error': {'message': "No file found with id 'file-Ssn6ZpKFKlTaFeNFXcxIYE8k'.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

And there you go!

We've fit our Assistant with an awesome Code Interpreter that lets our Assistant run code on our provided files!

### Creating an Assistant with a Function Calling Tool

Let's finally create an Assistant that utilizes the Function Calling API.

We'll start by creating a function that we wish to be called.

We'll utilize DuckDuckGo search to allow our Assistant to have the most up to date information!

In [105]:
!pip install -qU duckduckgo_search

In [106]:
from duckduckgo_search import DDGS

def duckduckgo_search(query):
  with DDGS() as ddgs:
    results = [r for r in ddgs.text(query, max_results=5)]
    return "\n".join(result["body"] for result in results)

Let's test our function to make sure it behaves as we expect it to.

In [107]:
duckduckgo_search("Who is the current prime minister of Canada?")

"Justin Trudeau (born December 25, 1971) is Canada's 23rd Prime Minister and the proud father of Xavier, Ella-Grace, and Hadrien. His vision of Canada is a country where everyone has a real and fair chance to succeed. His experiences as a teacher, father, leader, and advocate for youth have shaped his dedication to Canadians.\nConfederation in 1867, 23 prime ministers have formed 29 Canadian ministries [8] Justin Trudeau is the current prime minister, who took office on November 4, 2015, following the 2015 federal election, wherein his Liberal Party won a majority of seats.\nJustin Pierre James Trudeau-doh, troo- [ʒystɛ̃ pjɛʁ dʒɛms tʁydo]; born December 25, 1971) is a Canadian politician who has served as the 23rd prime minister of Canada since 2015 and the leader of the Trudeau was born in and attended Collège Jean-de-Brébeuf.\nJustin Trudeau (born December 25, 1971, Ottawa, Ontario, Canada) Canadian politician, prime minister of Canada (2015- ), leader of the Liberal Party (2013- ), 

Now we need to express how our function works in a way that is compatible with the OpenAI Function Calling API.

We'll want to provide a `JSON` object that includes what parameters we have, how to call them, and a short natural language description.

In [108]:
ddg_function = {
    "name" : "duckduckgo_search",
    "description" : "Answer non-technical questions. ",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "query" : {
                "type:" : "string",
                "description" : "The search query to use. For example: 'Who is the current Goalie of the Colorado Avalance?'"
            }
        },
        "required" : ["query"]
    }
}

####❓ Question

Why does the description key-value pair matter?

**Answer:** The description key-value pair describes what the function is designed to do. It matters because the LLM will use it to decide when to use the tool.

Now when we create our Assistant - we'll want to include the function description as a tool using the following format.

In [109]:
assistant = client.beta.assistants.create(
    name=name + " + Function Calling API",
    instructions=instructions,
    tools=[
        {"type": "function",
         "function" : ddg_function
        }
    ],
    model=model
)

We need to make a few modifications to our Assistant to include the ability to make calls to our local function and pass the results back to our Assistant for further generation.

In [110]:
import json

def wait_for_run_completion(thread_id, run_id):
    while True:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Current run status: {run.status}")
        if run.status in ['completed', 'failed', 'requires_action']:
            return run

def submit_tool_outputs(thread_id, run_id, tools_to_call):
    tool_output_array = []
    for tool in tools_to_call:
        output = None
        tool_call_id = tool.id
        function_name = tool.function.name
        function_args = tool.function.arguments

        if function_name == "duckduckgo_search":
            print("Consulting Duck Duck Go...")
            output = duckduckgo_search(query=json.loads(function_args)["query"])

        if output:
            tool_output_array.append({"tool_call_id": tool_call_id, "output": output})

    print(tool_output_array)

    return client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_output_array
    )

def print_messages_from_thread(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    for msg in messages:
        print(f"{msg.role}: {msg.content[0].text.value}")

def use_assistant(query, assistant_id, thread_id=None):
  thread = client.beta.threads.create()

  message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role="user",
      content=query,
  )

  print("Creating Assistant ")

  run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
  )

  print("Querying OpenAI Assistant Thread.")

  run = wait_for_run_completion(thread.id, run.id)

  if run.status == 'requires_action':
    run = submit_tool_outputs(thread.id, run.id, run.required_action.submit_tool_outputs.tool_calls)
    run = wait_for_run_completion(thread.id, run.id)

  print_messages_from_thread(thread.id)

  return thread.id

####❓ Question

Outline, in simple terms, what the `use_assistant` helper function is doing.

**Answer**:
The use_assistant helper function first creates a thread and a message for the user's query, and then creates a run on that thread. During the run, if the assistant determines that a tool needs to be used, the run will have a "required_action" state, and in the submit_tool_outputs method, we call the function that the assistant determined we needed. We return the response to OpenAI, and once the run is completed we get the final result back.

In [113]:
use_assistant("Who is the current leader of New Zealand?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_Mqn5J8WbwqXNVsKdwCfcNDK7', 'output': 'The incumbent prime minister, Christopher Luxon, leader of the New Zealand National Party, took office on 27 November 2023. [2] The prime minister (informally abbreviated to PM) ranks as the most senior government minister.\nChristchurch mosque shootings 2019 Whakaari / White Island eruption Dame Jacinda Kate Laurell Ardern [1] GNZM ɑːrɜːrjə--də ar-DURN [2] born 26 July 1980) is a New Zealand former politician who served as the 40th prime minister of New Zealand and leader of the Labour Party from 2017 to 2023.\nNew Zealand\'s Jacinda Ardern wins big after world-leading Covid-19 response "At a time where there\'s lots of instability and fear, I think most New Zealanders like the fact that she is clearly...\nJacinda Ardern (born July 26, 1980, Hamilton, New Zealand) New Zealand politician who in August 2017 bec

'thread_7lsogExlyuLXIFJU3lv0Hh7j'

## Wrapping it All Together

Now we can create an Assistant with all of the available tools and see how it responds to various queries!

In [114]:
assistant = client.beta.assistants.create(
    name=name + " + All Tools",
    instructions=instructions,
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {"type": "function", "function" : ddg_function}
    ],
    model=model,
    file_ids=[file_reference.id],
)

In [115]:
use_assistant("Who is the current prime minister of England?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_lcPJaqb0OhVQijKEnbjsCccQ', 'output': "Rishi Sunak This article is part of a series about Rishi Sunak Political positions Electoral history MP for Richmond (Yorks) Prime Minister of the United Kingdom Premiership Minister for the Union Ministry 2023 February reshuffle 2023 November reshuffle Industrial disputes postal workers strikes railway strikes NHS strikes 2022 autumn statement\nRishi Sunak has been the prime minister since 25 October 2022. [7] History Sir Robert Walpole is generally considered to have been the first person to hold the position of Prime Minister.\nLONDON — Rishi Sunak officially took over as Britain's 57th prime minister on Tuesday, vowing to fix the mistakes made by his predecessor, Liz Truss, and quickly worked to form a cabinet with an...\nRishi Sunak MP is the leader of the UK government and

'thread_fAsKKsDYOgkfdk8qCuIxtb1l'

In [116]:
use_assistant("Who is the author of the supplied file?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The author of the supplied file is Mary Wollstonecraft Shelley. She is the author of the eBook "Frankenstein; or, The Modern Prometheus"【5†source】.
user: Who is the author of the supplied file?


'thread_jNmXXitx5eekbVDXVzawlQum'

In [117]:
use_assistant("How many bytes is the provided file?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_vOiu1trWALuThjvgJUN21AG5', 'output': 'Kilobytes Kilobytes → Bytes 1 KB = 1000 Bytes (in decimal) 1 KB = 1024 Bytes (in binary) Bytes to KB KB to Bytes KB to MB KB to GB How many Bytes in a Kilobyte 1 Kilobyte is equal to 1000 bytes (decimal). 1 KB = 10 3 B in base 10 (SI). 1 Kilobyte is equal to 1024 bytes (binary). 1 KB = 2 10 B in base 2. Difference Between KB and B\nKilobytes to Bytes. Convert between the units (kB → B) or see the conversion table\nConvert 331 Kilobytes to Bytes (kB to Byte) with our conversion calculator and conversion tables. To convert 331 kB to Byte use direct conversion formula below. 331 kB = 338944 Byte. You also can convert 331 Kilobytes to other Storage (popular) units.\nThere are 0.0009765625 Kilobytes in a Byte. What is a Byte (B)? A Byte is a unit used to measure digital storage and i

'thread_COmqgCCeZA11gEajv94MMIaA'

####❓ Question

Notice that our response can go through multiple paths, given that:

What is "deciding" to use the tool?

**Answer:** The assistant (LLM) is deciding to use the tool. When it identifies that it needs us to take an action (like use our duckduckgo function), we are responsible for calling the function. However, the assistant is deciding which tool(s) to select based on the provided query.

### Adding JSON Mode for More Agentic Behaviour

Finally, we have the ability to select tools - all we need to do now is set up a process to allow us to create some kind of loop and make decisions about whether or not the response is complete or not.

We'll leverage the OpenAI completions end-endpoint with JSON mode to let us understand when we've adequately answered our user's question!

In [118]:
completed_template = \
"""
Does this response adequately answer the user's query?

Please return your response in JSON format - with key: "completed" and either True (if completed) or False (if not completed)

User Query:
{query}

Assistant Response:
{response}
"""

def is_complete(query, response):
  completed_response = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": completed_template.format(query=query, response=response),
          }
      ],
      model=model,
      response_format={"type" : "json_object"}
  )

  return completed_response

In [119]:
query = "How many bytes is the provided file?"

thread_id_for_response = use_assistant(query, assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The provided file contains 466,919 bytes.
user: How many bytes is the provided file?


Now we can observe JSON mode in action!

In [120]:
messages = client.beta.threads.messages.list(thread_id=thread_id_for_response)
response = messages.data[0].content[0].text.value
completed_flag = json.loads(is_complete(query, response).choices[0].message.content)

In [121]:
completed_flag

{'completed': True}

## 🚧 BONUS CHALLENGE 🚧:

Use the components we've constructed so far to build a loop that lets us continue to query the Assistant if the response is not completed!

In [ ]:
### YOUR CODE HERE

# Make Sure You Delete Resources

Make sure you delete all the resources you created!

This function will help you do so!

In [122]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)